In [1]:
import numpy as np
import pandas as pd
# import random as rnd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from patsy import dmatrices
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
# from sklearn.tree import export_graphviz
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

# import winsound
%matplotlib inline
import datetime

C:\Users\jakek\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
file = 'twenty'

In [3]:
df_classif = pd.read_csv('complete dfs/Complete compile/'+file+'.csv',low_memory= False)

In [4]:
# df2.replace('Monday',1,inplace=True)
# df2.replace('Tuesday',2,inplace=True)
# df2.replace('Wednesday',3,inplace=True)
# df2.replace('Thursday',4,inplace=True)
# df2.replace('Friday',5,inplace=True)
# df2.replace('Saturday',6,inplace=True)
# df2.replace('Sunday',7,inplace=True)

In [5]:
# df_classif = pd.concat([df_classif,df2])


In [6]:
df_classif.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,VehicleJourneyID,TimeFrame,StopID,Timestamp,AtStop,Instance,Direction,TimeToNextStop,Hour,Day,Avg,Summary,Temp,Rain,Wind_Speed,Route
0,338576,15893,15893,6383.0,2012-11-29,7353,1.354179e+15,0.0,1,0,120.0,8,4,64.152174,Scattered Clouds,-1.0,0.0,11.0,46A
1,178592,69003,69003,6918.0,2013-01-08,1016,1.357631e+15,1.0,1,0,39.0,7,2,47.257143,Mostly Cloudy,12.0,0.0,20.0,140
2,106456,106456,106456,6600.0,2013-01-18,790,1.358514e+15,1.0,1,1,101.0,12,5,120.956522,Light Rain,2.0,1.0,50.0,46A
3,72782,10764,10764,3481.0,2013-01-14,7401,1.358156e+15,1.0,1,0,205.0,9,1,226.586207,Mostly Cloudy,4.0,0.0,25.0,747
4,68160,68160,68160,2224.0,2012-11-27,1730,1.354028e+15,0.0,1,1,20.0,14,2,24.900000,Partly Cloudy,7.0,0.0,20.0,130


In [7]:
df_classif.replace('Clear',1,inplace=True)
df_classif.replace('Fog',2,inplace=True)
df_classif.replace('Light Drizzle',3,inplace=True)
df_classif.replace('Light Rain',4,inplace=True)
df_classif.replace('Light Rain Showers',5,inplace=True)
df_classif.replace('Light Small Hail Showers',6,inplace=True)
df_classif.replace('Light Snow Showers',7,inplace=True)
df_classif.replace('Mostly Cloudy',8,inplace=True)
df_classif.replace('Overcast',9,inplace=True)
df_classif.replace('Partly Cloudy',10,inplace=True)
df_classif.replace('Patches of Fog',11,inplace=True)
df_classif.replace('Rain',12,inplace=True)
df_classif.replace('Scattered Clouds',13,inplace=True)
df_classif.replace('Shallow Fog',14,inplace=True)

In [8]:
df_classif.Direction.unique()

array([0, 1], dtype=int64)

In [9]:
df_classif=df_classif[df_classif.Avg > 0]
df_classif=df_classif[df_classif.TimeToNextStop> 0]

In [44]:
def mins(x):
    x = x/1000000
    date = datetime.datetime.fromtimestamp(x).strftime("%H, %M")
    hour, minute = date.split(', ')
    date = hour+'.'+minute
    return(float(date))

In [45]:

df_classif['mins'] = df_classif.apply(lambda row: mins(row['Timestamp']), axis=1)

In [46]:
df_classif[['VehicleJourneyID','StopID','Timestamp','AtStop','Avg','TimeToNextStop','Day','Summary','Temp','Rain','Wind_Speed', 'Direction', 'mins']] = df_classif[['VehicleJourneyID','StopID','Timestamp','AtStop','Avg','TimeToNextStop','Day','Summary','Temp','Rain','Wind_Speed', 'Direction', 'mins']].astype(np.float32)
df_classif=df_classif.dropna()
df_classif.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,VehicleJourneyID,TimeFrame,StopID,Timestamp,AtStop,Instance,Direction,TimeToNextStop,Hour,Day,Avg,Summary,Temp,Rain,Wind_Speed,Route,mins
0,338576,15893,15893,6383.0,2012-11-29,7353.0,1.354178e+15,0.0,1,0.0,120.0,8,4.0,64.152176,13.0,-1.0,0.0,11.0,46A,8.41
1,178592,69003,69003,6918.0,2013-01-08,1016.0,1.357631e+15,1.0,1,0.0,39.0,7,2.0,47.257141,8.0,12.0,0.0,20.0,140,7.49
2,106456,106456,106456,6600.0,2013-01-18,790.0,1.358514e+15,1.0,1,1.0,101.0,12,5.0,120.956520,4.0,2.0,1.0,50.0,46A,12.57
3,72782,10764,10764,3481.0,2013-01-14,7401.0,1.358156e+15,1.0,1,0.0,205.0,9,1.0,226.586212,8.0,4.0,0.0,25.0,747,9.40
4,68160,68160,68160,2224.0,2012-11-27,1730.0,1.354028e+15,0.0,1,1.0,20.0,14,2.0,24.900000,10.0,7.0,0.0,20.0,130,14.51


In [47]:
# num = []
# for i in df_classif['Route'].unique():
#     try:
#         int(i)
#         num += [i]
#     except:
#         print(i, "is not int")

In [48]:
# testroute = df_classif.loc[df_classif['Route'] == num[0]]
# for i in num:
#     if i != num[0]:
#         testroute = pd.concat([testroute, df_classif.loc[df_classif['Route'] == i]]) 

In [49]:
df_cont_feat = df_classif[['Avg','Temp']]
df_dummies_broadband = df_classif[['StopID','AtStop','Day', 'mins']]
df_dummies_broadband.head()

,StopID,AtStop,Day,mins
0,7353.0,0.0,4.0,8.41
1,1016.0,1.0,2.0,7.49
2,790.0,1.0,5.0,12.57
3,7401.0,1.0,1.0,9.40
4,1730.0,0.0,2.0,14.51


In [50]:
X = pd.concat([df_cont_feat, df_dummies_broadband[['StopID','AtStop','Day', 'mins']]], axis =1)
y = df_classif.TimeToNextStop

In [51]:
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y = lab_enc.fit_transform(y)

In [52]:
rfr = RandomForestRegressor(n_estimators=15, max_features='auto', max_depth=50, oob_score=True, random_state=1, min_samples_split=100)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [54]:
rfr.fit(X_train, y_train)


C:\Users\jakek\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:723: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=100, min_weight_fraction_leaf=0.0,
           n_estimators=15, n_jobs=1, oob_score=True, random_state=1,
           verbose=0, warm_start=False)

In [55]:
rfr_predictions_test = rfr.predict(X_test)
df_true_vs_rfr_predicted_test = pd.DataFrame({'Actual': y_test, 'Predicte': rfr_predictions_test})
df_true_vs_rfr_predicted_test.head()

,Actual,Predicte
0,239,159.534784
1,17,23.292110
2,20,24.732061
3,62,68.099737
4,99,121.876480


In [22]:
print("Accuracy: ", metrics.accuracy_score(y_test, rfc_predictions_test))


NameError: name 'rfc_predictions_test' is not defined

In [56]:
new4=abs(df_true_vs_rfr_predicted_test.Actual-df_true_vs_rfr_predicted_test.Predicte)

total=0
for i in new4:
    if i<=20:
        total+=1
accuracy=(total/len(new4))
print(accuracy)

0.6882650471279577


In [57]:
pd.DataFrame({'feature': X_test.columns, 'importance':rfr.feature_importances_})

,feature,importance
0,Avg,0.915014
1,Temp,0.005971
2,StopID,0.026076
3,AtStop,0.022906
4,Day,0.006518
5,mins,0.023514


In [58]:
from sklearn.externals import joblib
joblib.dump(rfr, 'pickled rf/'+file+'extra time feature', compress=False)

['pickled rf/twentyextra time feature']

In [ ]:
import sys
sys.getsizeof(rfr)

In [59]:
X_test.head()

,Avg,Temp,StopID,AtStop,Day,mins
804025,150.000000,10.0,4348.0,1.0,4.0,16.18
2341592,30.000000,2.0,1196.0,0.0,3.0,18.16
604416,33.312500,11.0,410.0,0.0,1.0,12.21
1361175,71.000000,3.0,4276.0,0.0,4.0,9.24
537510,123.727272,6.0,3663.0,0.0,6.0,12.36


In [24]:
import winsound
while True:
    winsound.Beep(17000, 1000)

KeyboardInterrupt: 

In [ ]:
# df_classif = df_classif.sample(frac=0.4)

In [ ]:
train, test = train_test_split(df_classif, test_size=0.2)
train = train.sample(frac=0.4)

In [ ]:
train_X = train[['Avg','Temp', 'StopID','AtStop','Day','Hour']]
train_Y = train.TimeToNextStop
test_X = test[['Avg','Temp', 'StopID','AtStop','Day','Hour']]
test_Y = test.TimeToNextStop

In [ ]:
lab_enc = preprocessing.LabelEncoder()
train_Y = lab_enc.fit_transform(train_Y)
test_Y = lab_enc.fit_transform(test_Y)

In [ ]:
rfc.fit(train_X, train_Y)

In [ ]:
rfc_predictions_test = rfc.predict(test_X)

In [ ]:
df_true_vs_rfc_predicted_test = pd.DataFrame({'Actual': test_Y, 'Predicte': rfc_predictions_test})
df_true_vs_rfc_predicted_test.head()

In [ ]:
print("Accuracy: ", metrics.accuracy_score(test_Y, rfc_predictions_test))
new4=abs(df_true_vs_rfc_predicted_test.Actual-df_true_vs_rfc_predicted_test.Predicte)

total=0
for i in new4:
    if i<=20:
        total+=1
accuracy=(total/len(new4))
print(accuracy)

In [ ]:
joblib.dump(rfc, 'pickled rf/'+file+' 40percentoftrain.pkl', compress=True)